<a href="https://colab.research.google.com/github/ssegovba/identifying-deforestation/blob/main/notebooks/eda_satellite_imageryf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [ ]:
kaggle competitions download -c planet-understanding-the-amazon-from-space